In [ ]:
import os
import subprocess
import json
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import DBSCAN

# ✏️ Base 작업 디렉토리 및 서브폴더 목록
CUR_DIR = Path(r"D:\golfDataset\dataset\test")  # 현재 작업 루트 디렉토리
BALANCED_FOLDERS = ["balanced_true","false"]   # 처리할 두 가지 서브 디렉토리
FPS = 30

# OpenPose 설정
OPENPOSE_EXE  = Path(r"C:/openpose/openpose/bin/OpenPoseDemo.exe")
OPENPOSE_ROOT = OPENPOSE_EXE.parent.parent  # …/openpose 폴더
PAD_RATIO     = 0.10

# Torso Keypoint 인덱스 (Neck, MidHip, RShoulder, LShoulder, RootHip)
TORSO_IDXS = [1, 8, 11, 14, 17]
# Keypoint 이름 및 CSV 컬럼 생성
KP   = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist",
        "MidHip","RHip","RKnee","RAnkle","LHip","LKnee","LAnkle","REye","LEye",
        "REar","LEar","LBigToe","LSmallToe","LHeel","RBigToe","RSmallToe","RHeel"]
COLS = [f"{n}_{a}" for n in KP for a in ("x","y","c")]

# ---------- OpenPose 실행 함수 --------------------------------------
def run_openpose(video: Path, out_dir: Path):
    """OpenPose CLI를 통해 JSON으로 keypoints 추출"""
    out_dir.mkdir(parents=True, exist_ok=True)
    cmd = [str(OPENPOSE_EXE),
           "--video", str(video),
           "--write_json", str(out_dir),
           "--display", "0", "--render_pose", "0",
           "--number_people_max", "1",
           "--model_folder", str(OPENPOSE_ROOT / "models")]
    subprocess.run(cmd, check=True, cwd=OPENPOSE_ROOT)

# ---------- 주요 인물 박스 추출 함수 ---------------------------------
def main_person_boxes(json_dir: Path):
    centers, boxes = [], []
    for jf in sorted(json_dir.glob("*.json")):
        data = json.load(open(jf))
        people = data.get("people")
        if not people:
            continue
        kps = np.array(people[0]["pose_keypoints_2d"]).reshape(-1, 3)
        if kps[8, 2] < 0.10:  # MidHip confidence 확인
            continue
        cx, cy = kps[8, :2]
        valid = kps[:, 2] > 0.05
        xs, ys = kps[valid, 0], kps[valid, 1]
        centers.append([cx, cy])
        boxes.append([xs.min(), ys.min(), xs.max(), ys.max()])

    if not centers:
        return []
    centers = np.array(centers)
    labels = DBSCAN(eps=100, min_samples=5).fit_predict(centers)
    if (labels != -1).any():
        main_label = np.bincount(labels[labels != -1]).argmax()
    else:
        main_label = 0
    return [boxes[i] for i, lb in enumerate(labels) if lb == main_label]

# ---------- Union Box 계산 함수 ------------------------------------
def union_box(box_list):
    arr = np.array(box_list)
    x1, y1 = arr[:, :2].min(0)
    x2, y2 = arr[:, 2:].max(0)
    w, h    = x2 - x1, y2 - y1
    pad_w   = w * PAD_RATIO
    pad_h   = h * PAD_RATIO
    return int(x1 - pad_w), int(y1 - pad_h), int(w + 2 * pad_w), int(h + 2 * pad_h)

# ---------- FFmpeg Crop 함수 ----------------------------------------
def crop_video(src: Path, dst: Path, bbox):
    x, y, w, h = bbox
    cmd = ["ffmpeg", "-y", "-i", str(src),
           "-filter:v", f"crop={w}:{h}:{x}:{y}",
           "-pix_fmt", "yuv420p", str(dst)]
    subprocess.run(cmd, check=True)

# ---------- JSON → CSV 변환 함수 ------------------------------------
def json_dir_to_csv(json_dir: Path, csv_path: Path):
    rows = []
    for jf in sorted(json_dir.glob("*.json")):
        data = json.load(open(jf))
        people = data.get("people")
        if not people:
            rows.append([np.nan] * len(COLS))
        else:
            rows.append(people[0]["pose_keypoints_2d"])
    pd.DataFrame(rows, columns=COLS).to_csv(csv_path, index=False)

# ---------- 전체 파이프라인 함수 -----------------------------------
def preprocess_all(root_dir: Path):
    VIDEO_DIR      = root_dir / "video"
    CROP_VIDEO_DIR = root_dir / "crop_video"
    CROP_KP_DIR    = root_dir / "crop_keypoint"
    TMP_JSON_DIR   = root_dir / "_tmp_json"
    for d in [CROP_VIDEO_DIR, CROP_KP_DIR, TMP_JSON_DIR]:
        d.mkdir(parents=True, exist_ok=True)

    for vid in tqdm(sorted(VIDEO_DIR.glob("*.mp4")), desc=f"Processing {root_dir.name}"):
        name    = vid.stem
        raw_dir = TMP_JSON_DIR / f"raw_{name}"
        run_openpose(vid, raw_dir)

        boxes = main_person_boxes(raw_dir)
        if not boxes:
            print(f"⚠️  No valid person in {vid.name}")
            continue
        bbox = union_box(boxes)

        crop_mp4 = CROP_VIDEO_DIR / f"{name}_crop.mp4"
        crop_video(vid, crop_mp4, bbox)

        crop_dir  = TMP_JSON_DIR / f"crop_{name}"
        run_openpose(crop_mp4, crop_dir)

        csv_path = CROP_KP_DIR / f"{name}_crop.csv"
        json_dir_to_csv(crop_dir, csv_path)
        print(f"✅ {root_dir.name} / {name} saved")

    # 중간 JSON 삭제가 필요하다면 아래 주석 해제
    # shutil.rmtree(TMP_JSON_DIR)

# ---------- 스크립트 실행부 -----------------------------------------
if __name__ == "__main__":
    for folder in BALANCED_FOLDERS:
        root = CUR_DIR / folder
        preprocess_all(root)


Processing false:   1%|          | 1/148 [00:26<1:03:46, 26.03s/it]

✅ false / 20201123_General_030_DOS_A_M40_MM_070 saved


Processing false:   1%|▏         | 2/148 [00:55<1:07:38, 27.80s/it]

✅ false / 20201210_General_112_DOC_A_M20_MM_001 saved


Processing false:   2%|▏         | 3/148 [01:23<1:07:24, 27.89s/it]

...

✅ false / 20201210_General_114_NOC_A_M30_MM_075 saved


Processing false:  99%|█████████▉| 147/148 [1:14:55<00:24, 24.47s/it]

✅ false / 20201210_General_114_NOC_A_M30_MM_076 saved


Processing false: 100%|██████████| 148/148 [1:15:08<00:00, 30.47s/it]

✅ false / 20201210_General_114_NOC_A_M30_MM_077 saved
